In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 27.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.

In [4]:
import os
import gymnasium as gym
import panda_gym
from huggingface_sb3 import load_from_hub, package_to_hub
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

In [5]:
env_id = "PandaReachDense-v3"
env = gym.make(env_id)
s_size = env.observation_space.shape
a_size = env.action_space

In [6]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([-2.430506  , -7.92253   ,  0.93268335], dtype=float32)), ('desired_goal', array([ 7.1658964, -5.167857 ,  0.2818621], dtype=float32)), ('observation', array([-2.86151  ,  9.962164 ,  5.1103272,  9.110232 , -7.8655906,
        2.5791607], dtype=float32))])


In [7]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [ 0.25649846 -0.56894106 -0.6664344 ]


In [9]:
env = make_vec_env(env_id, n_envs=4)
env = VecNormalize(venv=env, norm_obs=True, norm_reward=True, clip_obs=10)

In [10]:
model = A2C(policy="MultiInputPolicy", env=env, verbose=1)

Using cuda device


In [11]:
model.learn(1_000_000)

Streaming output truncated to the last 5000 lines.
|    std                | 0.353    |
|    value_loss         | 0.000188 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.76     |
|    ep_rew_mean        | -0.224   |
|    success_rate       | 1        |
| time/                 |          |
|    fps                | 381      |
|    iterations         | 23800    |
|    time_elapsed       | 1246     |
|    total_timesteps    | 476000   |
| train/                |          |
|    entropy_loss       | -0.97    |
|    explained_variance | 0.978    |
|    learning_rate      | 0.0007   |
|    n_updates          | 23799    |
|    policy_loss        | -0.00482 |
|    std                | 0.351    |
|    value_loss         | 0.000164 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.76     |
|    ep_rew_mean        

In [12]:
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

In [13]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)
eval_env.render_mode = "rgb_array"
eval_env.training = False
eval_env.norm_reward = False
model = A2C.load("a2c-PandaReachDense-v3")
mean_reward, std_reward = evaluate_policy(model, eval_env)
print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward = -0.22 +/- 0.10


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [15]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [16]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"nithin04/a2c-{env_id}",
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmppeo4m4t6/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmppeo4m4t6/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmppeo4m4t6/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmppeo4m4t6/-step-0-to-step-1000.mp4
ℹ Pushing repo nithin04/a2c-PandaReachDense-v3 to the Hugging Face
Hub


policy.pth:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

...

vec_normalize.pkl:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

a2c-PandaReachDense-v3.zip:   0%|          | 0.00/111k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/nithin04/a2c-PandaReachDense-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/nithin04/a2c-PandaReachDense-v3/commit/9dffda30cf810e41d19a97f5ea5907502f77c1b1', commit_message='Initial commit', commit_description='', oid='9dffda30cf810e41d19a97f5ea5907502f77c1b1', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
env_id = "PandaPickAndPlace-v3"
env =  gym.make(env_id)
s_size = env.observation_space.shape
a_size = env.action_space
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (4,), float32)
Action Space Sample [ 8.3392435e-01 -5.3543027e-04 -5.9366733e-01  4.7784245e-01]
_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([-6.6003284,  5.8814225,  9.067385 ], dtype=float32)), ('desired_goal', array([ 0.67119724, -3.1305234 , -3.0292244 ], dtype=float32)), ('observation', array([ 7.6318007 , -5.4506354 ,  2.5456312 ,  1.4288236 , -1.3932426 ,
       -5.0827374 , -3.0183606 , -0.61092144,  0.05616569,  4.018951  ,
        1.0834404 , -2.8263025 ,  2.3845215 , -5.528405  ,  8.282388  ,
        8.325598  , -7.702781  , -8.302891  ,  7.8834577 ], dtype=float32))])


In [23]:
env = make_vec_env(env_id, n_envs=4)
env = VecNormalize(venv=env, norm_obs=True, norm_reward=True, clip_obs=True)
model = A2C(policy="MultiInputPolicy", env=env, verbose=1)
model.learn(10000)

Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 46.2     |
|    ep_rew_mean        | -46.1    |
|    success_rate       | 0.0976   |
| time/                 |          |
|    fps                | 298      |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -5.64    |
|    explained_variance | 0.946    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -1.62    |
|    std                | 0.992    |
|    value_loss         | 0.143    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 45.1     |
|    ep_rew_mean        | -45      |
|    success_rate       | 0.115    |
| time/                 |          |
|    fps                | 324      |
|    iterations     

In [24]:
model_name = "a2c-PandaPickAndPlace-v3_1000";
model.save(model_name)
env.save("vec_normalize_1000.pkl")
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("vec_normalize_1000.pkl", eval_env)
eval_env.training = False
eval_env.norm_reward = False
model = A2C.load(model_name)
mean_reward, std_reward = evaluate_policy(model, eval_env)
print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -45.00 +/- 15.00


In [2]:
# package_to_hub(
#     model=model,
#     model_name=f"a2c-{env_id}",
#     model_architecture="A2C",
#     env_id=env_id,
#     eval_env=eval_env,
#     repo_id=f"nithin04/a2c-{env_id}",
#     commit_message="Initial commit",
# )